In [17]:
import requests
import os
import pandas as pd
import numpy as np
import json
#from pandas import json_normalize
import sqlite3

#foursquare_api = "3u98iHboEuSwGOgUw7vIxg3C1EFb4yG+x7jWwwdRG1Q0=fsq"

In [18]:
import os
foursquare_api = os.environ['foursq_api']

KeyError: 'foursq_api'

In [36]:
#Send a request to Foursquare with a small radius (1000m) 
#with the location of your choice
"""url fields base"""

fsq_url_relevance = "https://api.foursquare.com/v3/places/search?query=restaurants&ll=44.38%2C-79.70&radius=1000&sort=RELEVANCE&limit=50"

"""using for popularity requests"""
fsq_url_poplt = "https://api.foursquare.com/v3/places/"
fsq_fields = "?fields=fsq_id%2Cpopularity"


fsq_headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api
}

In [37]:
fsq_relevn_res = requests.get(fsq_url_relevance, headers=fsq_headers)
fsq_relevn_json = fsq_relevn_res.json()

### helping function

In [38]:
def get_popolatiry(fsq_id):
    """return popularity data for the restaurant according to unique their fsq_id"""
    poplt_res = requests.get(fsq_url_poplt+fsq_id+fsq_fields, headers=fsq_headers)
    return poplt_res.json()

def write_json(respones_data, name_json):
    """make a json file if not exit with data file and name"""
    """name_json has to have prefix .json, and in string type"""
    with open(name_json, "w") as opf:
        json.dump(respones_data, opf)

def check_empty_data(clm):
    """check nan, empty list [], dict {} column with more than 40% within column"""
    count = 0
    nan_val = None
    for value in clm:
        #null_val = ["[]", "{}" "''", "NaN", "NULL"]
        if value == [] or value == {} or value == () or value == 'NaN':
            nan_val = value
            #print(value)
            count += 1
    #("null values percent: {}%".format(count/len(clm)*100))
    if (count/len(clm)*100) > 40:
        return True
    else:
        return False
    
def drop_na(df):
    """drop all columns with nan or empty list value more than 40% within column, using function check_empty_data"""
    nan_value = np.nan
    for col in df.columns:
        #bool_val, nul_val = check_empty_column(df[col])
        if check_empty_data(df[col]) == True:
            df.drop(col, axis=1, inplace=True)
            
def high_empty_rate(df):
    """the function will return table of isnull values and True False column with columns of data, with more than 40% of emtpty rows in column"""
    bool_list = []
    col_list = []
    is_nul = [val for val in df.isnull().sum()]
    for col in df.columns:
        TF_value = check_empty_data(df[col])
        col_list.append(col)
        bool_list.append(TF_value)
    list_of_tuples = list(zip(col_list, is_nul, bool_list))
    #isnull_check = pd.DataFrame(df_fsq_revln.isnull().sum())
    return pd.DataFrame(list_of_tuples, columns=['columnName','is-nul-check','40%-or higher-empty-data'])

### Exploring and Checking data

In [39]:
"""checking json file"""
fsq_relevn_json.keys()

dict_keys(['results', 'context'])

In [40]:
"""checking fsq_relevn_json file content"""
type(fsq_relevn_json), type(fsq_relevn_json['results']), type(fsq_relevn_json['context']), fsq_relevn_json['context'], fsq_relevn_json['results'][0]

(dict,
 list,
 dict,
 {'geo_bounds': {'circle': {'center': {'latitude': 44.38, 'longitude': -79.7},
    'radius': 1000}}},
 {'fsq_id': '4d2ef96479dd6ea83b848dd3',
  'categories': [{'id': 11057,
    'name': 'Caterer',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 226,
  'geocodes': {'main': {'latitude': 44.3814, 'longitude': -79.697968},
   'roof': {'latitude': 44.3814, 'longitude': -79.697968}},
  'link': '/v3/places/4d2ef96479dd6ea83b848dd3',
  'location': {'address': '96 Victoria St',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '96 Victoria St, Barrie ON L4N 2J1',
   'locality': 'Barrie',
   'postcode': 'L4N 2J1',
   'region': 'ON'},
  'name': "Fox's Catering",
  'related_places': {},
  'timezone': 'America/Toronto'})

In [41]:
"""getting relevent data"""
fsq_relevn_json = fsq_relevn_json['results']
fsq_poplt = [get_popolatiry(fsq['fsq_id']) for fsq in fsq_relevn_json]

In [42]:
fsq_relevn_json[23]

{'fsq_id': '55d6927e498ebcdc84cb2b8f',
 'categories': [{'id': 13031,
   'name': 'Burger Joint',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 931,
 'geocodes': {'main': {'latitude': 44.377342, 'longitude': -79.711353}},
 'link': '/v3/places/55d6927e498ebcdc84cb2b8f',
 'location': {'address': 'Dunlop St E',
  'country': 'CA',
  'cross_street': 'Mulcaster St',
  'formatted_address': 'Dunlop St E (Mulcaster St), Barrie ON',
  'locality': 'Barrie',
  'region': 'ON'},
 'name': 'The Marks',
 'related_places': {},
 'timezone': 'America/Toronto'}

In [43]:
#save json files
write_json(fsq_poplt ,"popularity_res_foursquare.json")
write_json(fsq_relevn_json,"relevn_res_foursquare.json")

In [44]:
#loading json files
df_fsq_poplt = pd.read_json("popularity_res_foursquare.json")
df_fsq_revln = pd.read_json("relevn_res_foursquare.json")

In [20]:
type(df_fsq_poplt), type(df_fsq_revln)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [21]:
df_fsq_revln.head(2)

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,4d2ef96479dd6ea83b848dd3,"[{'id': 11057, 'name': 'Caterer', 'icon': {'pr...",[],226,"{'main': {'latitude': 44.3814, 'longitude': -7...",/v3/places/4d2ef96479dd6ea83b848dd3,"{'address': '96 Victoria St', 'country': 'CA',...",Fox's Catering,{},America/Toronto
1,4b6eee17f964a5209cd12ce3,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",[],523,"{'main': {'latitude': 44.379593, 'longitude': ...",/v3/places/4b6eee17f964a5209cd12ce3,"{'address': '151 Bradford St', 'country': 'CA'...",Midway Diner,{},America/Toronto


In [61]:
def drop_non_postcode(df):
    """drop all columns without postcode"""
    for idx in df.index:
        #bool_val, nul_val = check_empty_column(df[col])
        if not "postcode" in df['location'][idx]:
            df.drop(idx, axis=0, inplace=True)

In [62]:
drop_non_postcode(df_fsq_revln)

In [63]:
len(df_fsq_revln)

24

In [85]:
def unpack_location(data):
    """unpacked location infos into list, for later use input to sqlite table"""
    #num = data.index
    location_list = []
    for num in data.index:
        for location in data['location']:
            #fsq_id = data['fsq_id'][num]
            details = pd.Series(location)[['address', 'locality','postcode', 'region', 'country']]
            fsqid_col = pd.Series([data_geo['fsq_id'][num]], index=['fsq_id'])
            location_list.append(pd.concat([fsqid_col, details]))
            
    return location_list

def unpack_categories(data):
    """unpacking the categories into dataframe for later, input to sqlite table"""
    cat_idx = data.set_index(['fsq_id']).categories.apply(pd.Series).stack().index
    df_catgrs = pd.DataFrame(data.set_index(['fsq_id']).categories.apply(pd.Series).stack().values.tolist(),index=cat_idx).reset_index().drop(['level_1', 'icon'], axis=1)
    return df_catgrs

In [86]:
df_fsq_revln.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 24],
           dtype='int64')

In [87]:
"""drop all high empty columns, according to function high_empty_rate return"""
drop_na(df_fsq_revln)

In [88]:
df_fsq_revln.head(2)

,fsq_id,categories,distance,geocodes,link,location,name,timezone
0,4d2ef96479dd6ea83b848dd3,"[{'id': 11057, 'name': 'Caterer', 'icon': {'pr...",226,"{'main': {'latitude': 44.3814, 'longitude': -7...",/v3/places/4d2ef96479dd6ea83b848dd3,"{'address': '96 Victoria St', 'country': 'CA',...",Fox's Catering,America/Toronto
1,4b6eee17f964a5209cd12ce3,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",523,"{'main': {'latitude': 44.379593, 'longitude': ...",/v3/places/4b6eee17f964a5209cd12ce3,"{'address': '151 Bradford St', 'country': 'CA'...",Midway Diner,America/Toronto


In [89]:
"""extracting category and location columns"""
data_categor = df_fsq_revln[['fsq_id', 'categories']]
data_geo = df_fsq_revln[['fsq_id', 'location']]

"""input to dataframe format"""
df_location = pd.DataFrame(unpack_location(df_fsq_revln))
df_categories = unpack_categories(data_categor)

In [90]:
df_location.head(3)

,fsq_id,address,locality,postcode,region,country
0,4d2ef96479dd6ea83b848dd3,96 Victoria St,Barrie,L4N 2J1,ON,CA
1,4d2ef96479dd6ea83b848dd3,151 Bradford St,Barrie,L4N 3B4,ON,CA
2,4d2ef96479dd6ea83b848dd3,29 Anne St S,Barrie,L4N 2C5,ON,CA


In [91]:
df_categories.head(3)

,fsq_id,id,name
0,4d2ef96479dd6ea83b848dd3,11057,Caterer
1,4b6eee17f964a5209cd12ce3,13049,Diner
2,4b6eee17f964a5209cd12ce3,13068,American Restaurant


### Get the popularity

In [92]:
"""drop columns with not relevant info, and columns that already be extacted"""
df_fsq_revln.drop(['geocodes', 'link', 'location', 'categories', 'distance'], axis=1, inplace=True)

In [93]:
type(df_fsq_poplt), type(df_fsq_revln), len(df_fsq_poplt), len(df_fsq_revln)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame, 25, 24)

In [94]:
"""merging two dataframes to one"""
df_places_popularity = pd.merge(left=df_fsq_revln, right=df_fsq_poplt, left_on='fsq_id', right_on='fsq_id')

In [95]:
df_places_popularity.head(5)

,fsq_id,name,timezone,popularity
0,4d2ef96479dd6ea83b848dd3,Fox's Catering,America/Toronto,0.986881
1,4b6eee17f964a5209cd12ce3,Midway Diner,America/Toronto,0.922274
2,5a70fb7a9411f2304935997a,Diamond Sushi,America/Toronto,0.942446
3,4f64d368e4b087553d3b8565,Tiffsbar,America/Toronto,0.885597
4,52cdd69b498e578c8ba767cd,Popeye's Louisiana Kitchen,America/Toronto,0.961490


In [96]:
df_places_popularity.columns, df_categories.columns, df_location.columns

(Index(['fsq_id', 'name', 'timezone', 'popularity'], dtype='object'),
 Index(['fsq_id', 'id', 'name'], dtype='object'),
 Index(['fsq_id', 'address', 'locality', 'postcode', 'region', 'country'], dtype='object'))

# Database FOURSQUARE

In [108]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, df, table):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        conn.commit()
        df.to_sql(table, conn, if_exists='replace', index=False)
    except Error as e:
        print(e)


In [99]:
#fsq_database = sqlite3.connect('square_database')
#fsq_conn = fsq_database.cursor()

In [117]:
def main():
    database = r"square_database.db"

    sql_popularity_table =  '''CREATE TABLE IF NOT EXISTS popularity_rate
                                ([popularity_id] INTEGER PRIMARY KEY AUTOINCREMENT,
                                [fsq_id] VARCHAR,
                                [name] TEXT,
                                [timezone] TEXT,
                                [popularity] DECIMAL
                                );'''
    sql_location_table = '''CREATE TABLE IF NOT EXISTS location
                                ([location_id] INTEGER PRIMARY KEY AUTOINCREMENT,
                                [fsq_id] VARCHAR,
                                [address] VARCHAR,
                                [locality] TEXT,
                                [postcode] VARCHAR,
                                [region] TEXT,
                                [country] TEXT
                            );'''
    sql_cat_table = '''CREATE TABLE IF NOT EXISTS categories
                             ([categories_id] INTEGER PRIMARY KEY AUTOINCREMENT,
                             [fsq_id] VARCHAR,
                             [id] VARCHAR,
                             [name] TEXT
                             );'''

    # create a database connection
    conn = create_connection(database)

    # create tables
    # tables 'popularity_rate', 'location', 'categories'
    if conn is not None:
        # create popularity table
        create_table(conn, sql_popularity_table,
                     df_places_popularity, 'popularity_rate')

        # create location table
        create_table(conn, sql_location_table,
                     df_location, 'location')
        
        # create catr table
        create_table(conn, sql_cat_table, df_categories, 'categories')
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()

In [118]:
from sqlite3 import connect

In [120]:
conn = connect('square_database.db')
pd.read_sql("""SELECT name, popularity
                FROM popularity_rate
                ORDER BY popularity DESC
                """, conn)

,name,popularity
0,Pie Wood Fired Pizza,0.988574
1,Fox's Catering,0.986881
2,Mary Browns,0.966991
3,Amaya Indian Room,0.964170
4,Popeye's Louisiana Kitchen,0.961490
5,The Italian Bakery,0.960220
6,Pizza Pizza,0.947383
7,Cafe Beans,0.945973
8,Diamond Sushi,0.942446
9,Subway,0.923402
